In [1]:
import csv
import cv2
import numpy as np
import datetime
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import os
import numpy as np
import matplotlib.image as mpimg

In [2]:
from keras.preprocessing import image
from keras.models import Model
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Lambda, Dropout
from keras.optimizers import Adam
from keras.layers import Convolution2D, ELU,Conv2D,MaxPooling2D
from keras.layers.convolutional import Cropping2D

In [3]:
img_loc = './Data/data/IMG/'
csv_file_loc = './Data/data/driving_log.csv'
data_loc = './Data/data/'

In [4]:
import pandas as pd
import os
data_df = pd.read_csv(csv_file_loc, names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

X = data_df[['center','left','right']].values
y = data_df['steering'].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 160, 320, 3
#IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 64,64, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
BATCH_SIZE  = 32
from utils import preprocess

def load_image(image_file):
    return mpimg.imread(os.path.join(data_loc, image_file.strip()))

def get_image_data(center, left, right, steering_angle):
    
    choice = np.random.choice(3)
    if choice == 0:
        return load_image(left), steering_angle + 0.2
    elif choice == 1:
        return load_image(right), steering_angle - 0.2
    return load_image(center), steering_angle


def flip_image(image, steering_angle):
    image = cv2.flip(image, 1)
    steering_angle = -float(steering_angle)
    return image, steering_angle


def batch_generator(image_paths, steering_angles):

    images = np.empty([BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    angles = np.empty(BATCH_SIZE)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center, left, right = image_paths[index]
            steering_angle = steering_angles[index]
            if steering_angle == 'steering':
                continue
            if center == 'center':
                continue
            steering_angle = float(steering_angle)
            image,steering_angle = get_image_data(center, left, right, steering_angle)
            # add the image and steering angle and flipped image and steering angle to the barch
            # convert the image YUV
            #image = preprocess(image)
            images[i] = cv2.cvtColor(image, cv2.COLOR_RGB2YUV) 
            angles[i] = steering_angle
            images[i+1], angles[i+1]= flip_image(image,steering_angle)
            i += 2
            if i >= BATCH_SIZE:
                break
        yield images, angles
    

In [10]:
model = Sequential()
model.add(Lambda(lambda image: image / 127.5 - 1., input_shape=(IMAGE_HEIGHT,IMAGE_WIDTH,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Conv2D(24, (5, 5), strides=(2, 2), activation="elu"))
model.add(Conv2D(36, (5, 5), strides=(2, 2), activation="elu"))
model.add(Conv2D(48, (5, 5), strides=(2, 2), activation="elu"))
model.add(Conv2D(64, (3, 3), activation="elu"))
model.add(Conv2D(64, (3, 3), activation="elu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1164))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
model.summary()
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer, loss='mse',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 31, 158, 24)       1824      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 77, 36)        21636     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 5, 37, 48)         43248     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 35, 64)         27712     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 1, 33, 64)        

In [11]:
model.fit(batch_generator(X_train, y_train),
                        steps_per_epoch = len(X_train)*4,
                        epochs = 3,
                        shuffle = True,
                        validation_data=batch_generator(X_valid, y_valid),
                        validation_steps=len(X_valid),
                        verbose=1)


Epoch 1/3
   24/25716 [..............................] - ETA: 2:41:00 - loss: 0.0558 - accuracy: 0.2018

KeyboardInterrupt: 

In [ ]:
model.save('model.h5')